<a href="https://colab.research.google.com/github/Ramanparjapati/goatai/blob/master/Generative_model_with_Torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd

# Step 1: Prepare the data
qa_pairs = [
    ("What is the capital of France?", "The capital of France is Paris."),
    ("Who wrote Romeo and Juliet?", "William Shakespeare wrote Romeo and Juliet."),
]
def load_qa_dataset(excel_file):
  data = pd.read_excel(excel_file)
  pa_pairs = [(row["Question"], row["Answer"]) for _, row in data.iterrows()]
  return qa_pairs

qa_pairs = load_qa_dataset("/content/s10-question-answer-pairs.xls")

# Step 2: Tokenization and vocabulary building
def build_vocab(qa_pairs):
    vocab = set()
    for question, answer in qa_pairs:
        vocab.update(question.split() + answer.split())
    return {word: i for i, word in enumerate(sorted(vocab))}

vocab = build_vocab(qa_pairs)
vocab_size = len(vocab)

# Step 3: Create a dataset class
class QADataset(Dataset):
    def _init_(self, qa_pairs, vocab):
        self.qa_pairs = qa_pairs
        self.vocab = vocab

    def _len_(self):
        return len(self.qa_pairs)

    def _getitem_(self, idx):
        question, answer = self.qa_pairs[idx]
        question_encoded = [self.vocab.get(word, 0) for word in question.split()]
        answer_encoded = [self.vocab.get(word, 0) for word in answer.split()]
        return torch.tensor(question_encoded), torch.tensor(answer_encoded)

def collate_fn(batch):
    questions, answers = zip(*batch)
    questions_padded = pad_sequence(questions, batch_first=True, padding_value=0)
    answers_padded = pad_sequence(answers, batch_first=True, padding_value=0)
    return questions_padded, answers_padded

# Step 4: Define the RNN model
class RNNModel(nn.Module):
    def _init_(self, vocab_size, embedding_dim, hidden_dim):
        super(RNNModel, self)._init_()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
        output, _ = self.rnn(packed)
        output, _ = pad_packed_sequence(output, batch_first=True)
        return self.fc(output)

# Step 5: Training function
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for questions, answers in train_loader:
            optimizer.zero_grad()
            lengths = torch.tensor([len(q[q != 0]) for q in questions])
            output = model(questions, lengths)
            loss = criterion(output.view(-1, vocab_size), answers.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Step 6: Generation function
def generate_answer(model, question, vocab, max_length=50):
    model.eval()
    rev_vocab = {i: word for word, i in vocab.items()}
    question_encoded = torch.tensor([[vocab.get(word, 0) for word in question.split()]])
    lengths = torch.tensor([question_encoded.size(1)])

    with torch.no_grad():
        output = model(question_encoded, lengths)
        _, predicted = output.max(2)

    answer = []
    for i in range(min(max_length, predicted.size(1))):
        next_word = rev_vocab[predicted[0][i].item()]
        if next_word == '.':
            break
        answer.append(next_word)

    return ' '.join(answer)

# Step 7: Set up and train the model
dataset = QADataset(qa_pairs, vocab)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

embedding_dim = 128
hidden_dim = 256
model = RNNModel(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # ignore padding index
optimizer = optim.Adam(model.parameters())
num_epochs = 100
train(model, train_loader, criterion, optimizer, num_epochs)

# Step 8: Use the model
while True:
    user_question = input("Ask a question (or type 'quit' to exit): ")
    if user_question.lower() == 'quit':
        break
    answer = generate_answer(model, user_question, vocab)
    print("Answer:", answer)
